In [1]:
import csv
import re
import time
import requests
import pandas as pd
import math

# sampling restaurants

rank 전처리

240page 있는 도시만 보류

240page 있는 도시중 random 50개 선택

review수에 따라 rank 하기

sampeling 50개 도시중에 식당

# recommend

In [1]:
with open('sampling_restaurants.csv', 'r',encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        city = row[0]
        name = row[1]
        link = row[5]

        url1 = f'https://www.yelp.com{link}/review_feed?rl=en&q=&sort_by=relevance_desc&start=0'
        print(name)
        try:
            res = requests.get(url=url1, headers=headers)
        except:
            time.sleep(500)
            res = requests.get(url=url1, headers=headers)
        try:
            count = re.findall('"count": (.*?)}', res.text)[0]
            page = math.ceil(int(count) / 10)
            print(page)
        except:
            page = 0

        for i in range(0, page):
            time.sleep(1)
            url = f'https://www.yelp.com{link}/review_feed?rl=en&q=&sort_by=relevance_desc&start={i * 10}'
            print(url)
            try:
                r = requests.get(url=url, headers=headers)
            except:
                time.sleep(500)
                r = requests.get(url=url, headers=headers)
            
            r = requests.get(url=url, headers=headers)
            tree = etree.HTML(r.text)
            al = re.findall('"userId"([\S\s]*?)]}[,\]]', r.text)
            print(len(al))

            if len(al) == 0:
                time.sleep(500)
                r = requests.get(url=url, headers=headers)
                al = re.findall('"userId"([\S\s]*?)]}[,\]]', r.text)
                print(len(al))
                if len(al) == 0:
                    break

            for a in al:
                # print(a)
                markupDisplayName = re.findall('"markupDisplayName": "([\S\s]*?)",', a)[0].strip()  # 用户名
                displayLocation = re.findall('"displayLocation": "([\S\s]*?)",', a)[0].strip()  # 地址
                src = re.findall('"src": "([\S\s]*?)",', a)[0].strip()  # 头像
                friendCount = re.findall('"friendCount": ([\S\s]*?),', a)[0].strip()  # 朋友数
                reviewCount = re.findall('"reviewCount": ([\S\s]*?),', a)[0].strip()  # 评论数
                photoCount = re.findall('"photoCount": ([\S\s]*?),', a)[0].strip()  # 评论数
                text = re.findall('"text": "([\S\s]*?)",', a)[0].strip()
                localizedDate = re.findall('"localizedDate": "([\S\s]*?)",', a)[0].strip()
                rating = re.findall('"rating": ([\S\s]*?),', a)[0].strip()

                #print(markupDisplayName, displayLocation, src, friendCount, reviewCount, photoCount, text, localizedDate, rating)

                new_save_obj['city'].append(city)
                new_save_obj['name'].append(name)
                new_save_obj['markupDisplayName'].append(markupDisplayName)
                new_save_obj['displayLocation'].append(displayLocation)
                new_save_obj['src'].append(src)
                new_save_obj['friendCount'].append(friendCount)
                new_save_obj['reviewCount'].append(reviewCount)
                new_save_obj['photoCount'].append(photoCount)
                new_save_obj['text'].append(text)
                new_save_obj['localizedDate'].append(localizedDate)
                new_save_obj['rating'].append(rating)

            df = pd.DataFrame(new_save_obj)
            df.to_csv('recommend_reviews.csv', index=False)

NameError: name 'csv' is not defined

In [ ]:
recommend_reviews =pd.read_csv('recommend_reviews.csv')

In [9]:
# import pandas as pd

# # 读取数据集
# df = pd.read_csv('recommend_reviews.csv')

# 使用drop_duplicates方法删除具有相同值的重复行，保留第一个出现的行
df_cleaned = df.drop_duplicates(subset=['name', 'markupDisplayName', 'localizedDate'])

# 保存清理后的数据集
df_cleaned.to_csv('recommend_reviews.csv', index=False)

In [10]:
df_cleaned.shape

(438453, 11)

# non_recommend

In [16]:
non_recommend_reviews =pd.read_csv('不推荐评论.csv')

In [17]:
non_recommend_reviews.shape

(44386, 11)

In [18]:
table = non_recommend_reviews.head()
display(table)

,city,r_name,username,text,address,star,src,time,friend_count,review_count,photo_count
0,Alameda,Jong Ga House,cher b.,First don't come here !!! Very bad food ...The...,"Berkeley, CA",1.0,https://s3-media0.fl.yelpcdn.com/photo/rxHTvp3...,8/23/2017,29,21,NaN
1,Alameda,Jong Ga House,Khristy W.,This place smells bad and the service is bad. ...,"Richmond, CA",1.0,https://s3-media0.fl.yelpcdn.com/photo/1BdGk-s...,2/5/2014,218,8,16.0
2,Alameda,Jong Ga House,Brittany T.,I came to the spot because I heard of all the ...,"Oakland, CA",2.0,https://s3-media0.fl.yelpcdn.com/assets/srv0/y...,11/5/2015,0,31,NaN
3,Alameda,Jong Ga House,Daniel M.,Lots banchan but nothing to eat. Kimchee was t...,"Lafayette, CA",2.0,https://s3-media0.fl.yelpcdn.com/photo/-ciyPmr...,3/5/2013,2,4,NaN
4,Alameda,Jong Ga House,Annie E.,First i want to say thanks for the delicious s...,"Los Angeles, CA",3.0,https://s3-media0.fl.yelpcdn.com/photo/mNN0Icd...,8/28/2018,49,5,1.0


In [19]:
na_count = non_recommend_reviews['photo_count'].isna().sum()
print("The number of NaN:", na_count)

The number of NaN: 29619


In [20]:
non_recommend_reviews['photo_count'].fillna(0, inplace=True)

In [21]:
description = non_recommend_reviews.describe()
display(description)

,star,friend_count,review_count,photo_count
count,44386.000000,44386.000000,44386.000000,44386.000000
mean,4.188122,36.542919,21.212409,23.636755
std,1.287816,142.380529,80.494830,888.157365
min,1.000000,0.000000,0.000000,0.000000
25%,4.000000,0.000000,2.000000,0.000000
50%,5.000000,0.000000,5.000000,0.000000
75%,5.000000,10.000000,14.000000,2.000000
max,5.000000,4999.000000,5915.000000,113799.000000
